# Playlist Sorter Notebook

This notebook implements a `PlaylistSorter` class to sort music playlists based on harmonic mixing (Camelot keys), BPM, and energy.

**Features:**
*   Loads playlist data from a CSV file.
*   Requires columns: `Track`, `Camelot`, `BPM`, `Energy`.
*   Builds a Camelot wheel neighbor map.
*   Finds a suitable opener track based on energy (and optionally popularity).
*   Sorts the playlist by finding the best transitions based on Camelot key compatibility and BPM similarity.
*   Exports the sorted playlist to a new CSV file.
*   Optionally prints the Camelot and BPM transition paths.

## Setting Up a Virtual Environment (Recommended)

Before running this notebook and installing packages, it's best practice to create an isolated Python environment using `venv`. This prevents conflicts between project dependencies.

**Steps:**

1.  **Open your terminal or command prompt.**
2.  **Navigate to your project directory** (where you saved this `.ipynb` file).
3.  **Create the virtual environment:**
    ```bash
    python -m venv venv 
    ```
    (You can replace `venv` with any name you prefer for the environment folder).
4.  **Activate the virtual environment:**
    *   **Windows (Command Prompt):** `venv\Scripts\activate`
    *   **Windows (PowerShell):** `venv\Scripts\Activate.ps1` (You might need to adjust execution policy: `Set-ExecutionPolicy Unrestricted -Scope Process`)
    *   **macOS/Linux (bash/zsh):** `source venv/bin/activate`
    
    You'll know it's active because your terminal prompt will usually show the environment name (e.g., `(venv) Your-Computer:project_folder $`).
5.  **Install required packages:**
    ```bash
    pip install pandas numpy jupyterlab # Or 'pip install pandas numpy notebook' for classic notebook
    ```
6.  **Start JupyterLab (or Notebook):**
    ```bash
    jupyter lab
    # or
    # jupyter notebook 
    ```
    This will open Jupyter in your browser, running with the packages installed in your virtual environment.
7.  **Deactivate (when done):** Simply type `deactivate` in the terminal.

In [1]:
import pandas as pd
import argparse
from typing import List, Dict, Optional
import csv
import os # Added for file existence check in notebook example

## The PlaylistSorter Class

In [2]:
class PlaylistSorter:
    """Tool for sorting playlists based on Camelot keys and other audio features."""
    
    def __init__(self, input_file: str):
        """Initialize with input CSV file containing playlist data."""
        self.tracks = self._load_tracks(input_file)
        if self.tracks is not None: # Only build neighbors if loading succeeded
            self.camelot_neighbors = self._build_camelot_neighbors()
        else:
             # Raise an exception or handle the error appropriately
             # For notebook usage, we might let it proceed and fail later, 
             # or raise an exception here.
             # Raising an exception is cleaner if _load_tracks guarantees failure exit.
             # Since _load_tracks uses sys.exit, we might not reach here in script mode,
             # but in notebook mode, sys.exit might just kill the kernel.
             # Let's assume _load_tracks returns None on failure for notebook context.
             raise ValueError(f"Failed to load tracks from {input_file}. Cannot initialize Sorter.")
        
    def _load_tracks(self, input_file: str) -> Optional[pd.DataFrame]:
        """Load tracks from a CSV file."""
        try:
            if not os.path.exists(input_file):
                print(f"Error: Input file not found at '{input_file}'")
                # In a notebook, sys.exit is disruptive. Return None or raise Exception.
                # Returning None allows the caller (__init__) to handle it.
                return None 
                
            df = pd.read_csv(input_file)
            required_columns = ["Track", "Camelot", "BPM", "Energy"]
            
            # Check if all required columns exist
            missing_columns = [col for col in required_columns if col not in df.columns]
            if missing_columns:
                print(f"Error: Missing required columns: {', '.join(missing_columns)}")
                if "Camelot" not in df.columns and "Key" in df.columns:
                    print("Note: 'Camelot' column is missing but 'Key' is present. Consider converting Key to Camelot format.")
                # In a notebook, sys.exit is disruptive. Return None or raise Exception.
                return None
                
            # Attempt to convert relevant columns to numeric, coercing errors
            for col in ['BPM', 'Energy']:
                 if col in df.columns:
                     df[col] = pd.to_numeric(df[col], errors='coerce')
            
            # Handle potential NaN values introduced by coercion or already present
            if df['BPM'].isnull().any() or df['Energy'].isnull().any():
                print("Warning: Found non-numeric or missing values in BPM or Energy columns. Rows with issues:")
                print(df[df['BPM'].isnull() | df['Energy'].isnull()])
                # Option 1: Drop rows with NaN in essential columns
                df.dropna(subset=['BPM', 'Energy'], inplace=True)
                print(f"Removed rows with missing BPM/Energy. {len(df)} tracks remaining.")
                # Option 2: Fill with a default (e.g., median), but dropping is safer for sorting
                # df['BPM'].fillna(df['BPM'].median(), inplace=True)
                # df['Energy'].fillna(df['Energy'].median(), inplace=True)

            if df.empty:
                print("Error: No valid tracks remaining after handling missing/invalid data.")
                return None

            # Ensure Camelot is string type
            if 'Camelot' in df.columns:
                 df['Camelot'] = df['Camelot'].astype(str)

            return df
        except Exception as e:
            print(f"Error loading tracks from {input_file}: {e}")
            # In a notebook, sys.exit is disruptive. Return None or raise Exception.
            return None
    
    def _build_camelot_neighbors(self) -> Dict[str, Dict[str, List[str]]]:
        """Build a dictionary of neighboring keys on the Camelot wheel."""
        neighbors = {}
        
        # For each number 1-12
        for num in range(1, 13):
            # For both A (minor) and B (major)
            for letter in ['A', 'B']:
                key = f"{num}{letter}"
                
                # Same number, different letter (e.g., 1A to 1B)
                related_letter = 'B' if letter == 'A' else 'A'
                related_by_letter = f"{num}{related_letter}"
                
                # Same letter, +1 number (wrapping from 12 to 1)
                next_num = 1 if num == 12 else num + 1
                related_next = f"{next_num}{letter}"
                
                # Same letter, -1 number (wrapping from 1 to 12)
                prev_num = 12 if num == 1 else num - 1
                related_prev = f"{prev_num}{letter}"
                
                # Add all neighbors
                # Primary neighbors (closest matches)
                primary = [related_by_letter, related_next, related_prev]
                # Secondary neighbors (still work but slightly less perfect)
                secondary = []
                
                # Add the +1/-1 for the related letter
                next_num_related = 1 if num == 12 else num + 1
                prev_num_related = 12 if num == 1 else num - 1
                secondary.append(f"{next_num_related}{related_letter}")
                secondary.append(f"{prev_num_related}{related_letter}")
                
                # Add the +2/-2 for the same letter
                # Corrected modulo arithmetic for wrapping
                next_num_2 = (num % 12) + 1 # +1 step
                next_num_2 = (next_num_2 % 12) + 1 # +2 step
                
                prev_num_2 = (num - 2 + 12) % 12 + 1 # -1 step
                prev_num_2 = (prev_num_2 - 2 + 12) % 12 + 1 # -2 step
                
                secondary.append(f"{next_num_2}{letter}")
                secondary.append(f"{prev_num_2}{letter}")
                
                # Remove duplicates just in case logic generated some (e.g., at wraps)
                primary = list(dict.fromkeys(primary))
                secondary = list(dict.fromkeys(secondary))
                all_neighbors = list(dict.fromkeys(primary + secondary))
                
                neighbors[key] = {
                    'primary': primary,
                    'secondary': secondary,
                    'all': all_neighbors
                }
                
        return neighbors
    
    def get_track_by_index(self, index: int) -> Optional[Dict]:
        """Get track information as a dictionary by index."""
        if self.tracks is None:
            print("Error: Tracks not loaded.")
            return None
        if 0 <= index < len(self.tracks):
            return self.tracks.iloc[index].to_dict()
        print(f"Error: Index {index} out of bounds.")
        return None
        
    def find_best_opener(self) -> Optional[int]:
        """Find the best opener track based on energy and popularity."""
        if self.tracks is None or self.tracks.empty:
            print("Error: No tracks available to find an opener.")
            return None
            
        # Prioritize tracks with higher energy and popularity
        # Use .get('Popularity', pd.Series(0, index=self.tracks.index)) for robustness
        popularity_col = self.tracks.get('Popularity', pd.Series(0, index=self.tracks.index))
        # Ensure popularity is numeric, default to 0 if not
        popularity_col = pd.to_numeric(popularity_col, errors='coerce').fillna(0)
        
        # Ensure Energy is present and numeric (already handled in _load_tracks, but double-check)
        if 'Energy' not in self.tracks or not pd.api.types.is_numeric_dtype(self.tracks['Energy']):
             print("Error: 'Energy' column is missing or not numeric.")
             # Fallback: maybe just return the first track index?
             return self.tracks.index[0] if not self.tracks.empty else None
             
        # Normalize popularity (0-100) to 0-1 scale
        max_popularity = popularity_col.max()
        normalized_popularity = popularity_col / 100.0 if max_popularity > 0 else popularity_col
        
        # Normalize energy (assuming it's often 0-1, but check range just in case)
        # If energy is not 0-1, adjust normalization. Assuming it is for now.
        energy_col = self.tracks['Energy']
        
        self.tracks['opener_score'] = (
            energy_col * 0.6 + 
            normalized_popularity * 0.4
        )
        
        # idxmax returns the index label
        best_opener_index_label = self.tracks['opener_score'].idxmax()
        # We need the positional index for list operations later
        best_opener_positional_index = self.tracks.index.get_loc(best_opener_index_label)
        
        # Clean up the temporary column
        # del self.tracks['opener_score'] 
        # Keep score for debugging if needed
        
        print(f"Selected opener: Track '{self.tracks.loc[best_opener_index_label, 'Track']}' (Index: {best_opener_positional_index}) with score {self.tracks.loc[best_opener_index_label, 'opener_score']:.2f}")
        return best_opener_positional_index
    
    def find_best_transition(self, current_track_dict: Dict, remaining_indices: List[int], 
                             allow_secondary: bool = False) -> Optional[int]:
        """Find the best next track based on Camelot compatibility and BPM."""
        if self.tracks is None:
             print("Error: Tracks not loaded.")
             return None
             
        current_camelot = current_track_dict.get('Camelot')
        current_bpm = current_track_dict.get('BPM')
        
        if not current_camelot or not current_bpm or current_camelot not in self.camelot_neighbors:
            print(f"Warning: Invalid current track data (Camelot: {current_camelot}, BPM: {current_bpm}). Cannot find transition.")
            # Fallback: maybe return the first remaining index?
            return remaining_indices[0] if remaining_indices else None
        
        # Get compatible keys
        compatible_keys = self.camelot_neighbors[current_camelot]['primary']
        if allow_secondary:
            compatible_keys += self.camelot_neighbors[current_camelot]['secondary']
        # Ensure unique keys
        compatible_keys = list(dict.fromkeys(compatible_keys))
        
        best_match_idx = None
        best_score = -1
        
        # Use positional indices for iloc
        for positional_idx in remaining_indices:
            # Get track data using positional index
            next_track = self.tracks.iloc[positional_idx].to_dict()
            next_camelot = next_track.get('Camelot')
            next_bpm = next_track.get('BPM')
            
            if not next_camelot or pd.isna(next_bpm):
                # Skip tracks with missing essential data
                continue 
            
            # Check key compatibility
            is_compatible = False
            key_match_score = 0.0
            if next_camelot == current_camelot:
                is_compatible = True
                key_match_score = 1.0
            elif next_camelot in self.camelot_neighbors[current_camelot]['primary']:
                is_compatible = True
                key_match_score = 0.8
            elif allow_secondary and next_camelot in self.camelot_neighbors[current_camelot]['secondary']:
                is_compatible = True
                key_match_score = 0.5
                
            if not is_compatible:
                continue
            
            # Calculate BPM match (closer is better, normalized to a 0-1 scale)
            bpm_diff = abs(next_bpm - current_bpm)
            # Consider a reasonable max diff for normalization, e.g., 30 BPM? 50 seems large.
            # Let's use 30: 0 diff = 1.0, 30+ diff = 0.0
            max_relevant_bpm_diff = 30.0 
            bpm_match_score = max(0, 1 - (bpm_diff / max_relevant_bpm_diff))
            
            # Combined score (weights can be tuned)
            # Give key slightly more weight than BPM
            match_score = (key_match_score * 0.6) + (bpm_match_score * 0.4)
            
            if match_score > best_score:
                best_score = match_score
                best_match_idx = positional_idx # Store the positional index
        
        return best_match_idx
    
    def create_sorted_playlist(self, start_index: Optional[int] = None) -> Optional[List[int]]:
        """Create a sorted playlist starting from the specified track (using positional index)."""
        if self.tracks is None or self.tracks.empty:
            print("Error: No tracks loaded to create a playlist.")
            return None
            
        num_tracks = len(self.tracks)
        if num_tracks == 0:
            print("Playlist is empty.")
            return []
            
        # Validate start_index if provided
        if start_index is not None:
            if not (0 <= start_index < num_tracks):
                print(f"Error: Provided start_index {start_index} is out of bounds (0-{num_tracks-1}). Finding best opener instead.")
                start_index = None
        
        # If no valid start index, find a good opener
        if start_index is None:
            start_index = self.find_best_opener()
            if start_index is None: # Could happen if find_best_opener failed
                 print("Error: Could not determine a starting track. Aborting sort.")
                 return None
            
        # Use positional indices throughout
        sorted_indices = [start_index]
        remaining_indices = list(range(num_tracks))
        remaining_indices.remove(start_index)
        
        current_track_positional_idx = start_index
        
        # Continue while we have tracks remaining
        while remaining_indices:
            current_track_dict = self.get_track_by_index(current_track_positional_idx)
            if current_track_dict is None:
                 print(f"Error: Could not retrieve data for track index {current_track_positional_idx}. Aborting.")
                 return None # Or handle differently
                 
            # Try to find a good transition using primary neighbors
            next_index = self.find_best_transition(current_track_dict, remaining_indices, allow_secondary=False)
            transition_type = "Primary Key Match"
            
            # If no primary match found, try secondary neighbors
            if next_index is None:
                next_index = self.find_best_transition(current_track_dict, remaining_indices, allow_secondary=True)
                transition_type = "Secondary Key Match"
            
            # If still no harmonic match, find the closest BPM among remaining
            if next_index is None:
                transition_type = "Fallback: Closest BPM"
                min_bpm_diff = float('inf')
                fallback_idx = None
                current_bpm = current_track_dict.get('BPM')
                if current_bpm is not None:
                    for idx in remaining_indices:
                        next_track_bpm = self.tracks.iloc[idx].get('BPM')
                        if next_track_bpm is not None:
                            diff = abs(next_track_bpm - current_bpm)
                            if diff < min_bpm_diff:
                                min_bpm_diff = diff
                                fallback_idx = idx
                
                # If we found a closest BPM track, use it. Otherwise, just take the first remaining.
                if fallback_idx is not None:
                     next_index = fallback_idx
                elif remaining_indices: # Ensure list isn't empty
                     next_index = remaining_indices[0]
                else:
                     # Should not happen if loop condition is correct, but defensively:
                     break 
            
            # Debugging print (optional)
            print(f"Transition: {current_track_dict['Track']} -> {self.get_track_by_index(next_index)['Track']} (via {transition_type})")
            
            sorted_indices.append(next_index)
            remaining_indices.remove(next_index)
            current_track_positional_idx = next_index
        
        return sorted_indices
    
    def export_sorted_playlist(self, sorted_indices: List[int], output_file: str) -> None:
        """Export the sorted playlist (using positional indices) to a CSV file."""
        if self.tracks is None:
             print("Error: Tracks not loaded. Cannot export.")
             return
        if not sorted_indices:
            print("Warning: No sorted indices provided or playlist is empty. Nothing to export.")
            return
            
        try:
            # Use positional indices with iloc
            sorted_tracks_df = self.tracks.iloc[sorted_indices].copy()
            
            # Overwrite the existing '#' column or create it if it doesn't exist
            # This ensures the '#' column reflects the NEW sorted order.
            sorted_tracks_df["#"] = range(1, len(sorted_tracks_df) + 1)

            # If the original '#' column was not the first one, ensure it is now.
            if "#" in sorted_tracks_df.columns:
                cols = sorted_tracks_df.columns.tolist()
                if cols[0] != "#":  # Check if '#' is already the first column
                    cols.remove("#")
                    cols.insert(0, "#")
                    sorted_tracks_df = sorted_tracks_df[cols]
            
            # Optionally remove temporary columns like 'opener_score'
            if 'opener_score' in sorted_tracks_df.columns:
                 sorted_tracks_df = sorted_tracks_df.drop(columns=['opener_score'])
                 
            sorted_tracks_df.to_csv(output_file, index=False, quoting=csv.QUOTE_NONNUMERIC)
            print(f"Sorted playlist exported to {output_file}")
        except Exception as e:
            print(f"Error exporting playlist to {output_file}: {e}")
        
    def print_transition_path(self, sorted_indices: List[int]) -> None:
        """Print the Camelot and BPM transition paths for visualization."""
        if self.tracks is None:
             print("Error: Tracks not loaded. Cannot print path.")
             return
        if not sorted_indices:
            print("Playlist is empty. No path to print.")
            return
            
        camelot_path = []
        bpm_path = []
        track_names = []
        
        print("\n--- Transition Path ---")
        print(" # | Track Name                      | Camelot | BPM")
        print("--|---------------------------------|---------|-----")
        
        max_name_len = 30 # Max length for track name column
        
        for i, idx in enumerate(sorted_indices):
            track = self.get_track_by_index(idx)
            if track:
                cmlt = track.get('Camelot', 'N/A')
                bpm = track.get('BPM', 'N/A')
                name = track.get('Track', 'Unknown Track')
                
                camelot_path.append(str(cmlt))
                bpm_path.append(f"{bpm:.0f}" if isinstance(bpm, (int, float)) and not pd.isna(bpm) else str(bpm))
                track_names.append(name)
                
                # Truncate long names for display
                display_name = (name[:max_name_len-3] + '...') if len(name) > max_name_len else name
                print(f"{i+1:<2} | {display_name:<{max_name_len}} | {str(cmlt):<7} | {bpm_path[-1]}")
            else:
                print(f"{i+1:<2} | Error retrieving track index {idx}")
                camelot_path.append("ERR")
                bpm_path.append("ERR")
                track_names.append("Error")

        print("\nCamelot Sequence:")
        print(" → ".join(camelot_path))
        
        print("\nBPM Sequence:")
        print(" → ".join(bpm_path))
        print("-----------------------")

## Command-Line Main Function (for reference)

This is the `main` function used when running the script from the command line with `argparse`. We won't call this directly in the notebook, but it's here for completeness.

In [3]:
def main():
    parser = argparse.ArgumentParser(description='Sort a playlist based on Camelot wheel and audio features')
    parser.add_argument('input_file', help='Input CSV file with playlist data')
    parser.add_argument('-o', '--output', default='sorted_playlist.csv', help='Output CSV file')
    # Ensure start index is treated as positional index (0-based)
    parser.add_argument('-s', '--start', type=int, help='Track positional index (0-based) to start with. If omitted, finds best opener.') 
    parser.add_argument('-v', '--verbose', action='store_true', help='Show transition paths')
    
    # In a script, sys.exit() is fine. In a notebook, it can kill the kernel.
    # The class methods now return None or raise exceptions on critical errors.
    try:
        args = parser.parse_args()
        
        sorter = PlaylistSorter(args.input_file)
        # Check if sorter initialized correctly (tracks loaded)
        if sorter.tracks is None:
             print("Initialization failed. Exiting.")
             # sys.exit(1) # Avoid sys.exit in notebook context if possible
             return # Exit the main function
             
        sorted_indices = sorter.create_sorted_playlist(args.start)
        
        if sorted_indices is not None:
            sorter.export_sorted_playlist(sorted_indices, args.output)
            if args.verbose:
                sorter.print_transition_path(sorted_indices)
        else:
             print("Failed to create sorted playlist.")
             
    except Exception as e:
         # Catch potential errors during argument parsing or execution
         print(f"An error occurred in main: {e}")
         # sys.exit(1)

# Standard script execution guard (won't run in notebook cell execution)
# if __name__ == "__main__":
#     main()

## Generating the Input CSV Data

This script requires a CSV file containing track data, including `Track`, `Camelot`, `BPM`, and `Energy` columns.

A convenient way to get this data for a Spotify playlist is using the **Spotify Playlist Analyzer** tool available at:

[https://songdata.io/spotify-playlist-analysis](https://songdata.io/spotify-playlist-analysis)

**How to use it:**

1.  Go to the website.
2.  Paste the link (URL) of your Spotify playlist into the input field.
3.  Click "Analyze Playlist".
4.  The tool will process the playlist and display a table with various audio features, including Key, Camelot, BPM, Energy, Danceability, Popularity, etc.
5.  **Convert the HTML table to CSV:** The easiest way is usually to:
    *   Select the entire table content on the webpage.
    *   Copy it (Ctrl+C or Cmd+C).
    *   Paste it into a spreadsheet program (like Google Sheets, Microsoft Excel, LibreOffice Calc).
    *   Save or export the sheet as a CSV file (e.g., `your_playlist_data.csv`).

Make sure the resulting CSV file has the necessary column headers (`Track`, `Camelot`, `BPM`, `Energy`). You might need to adjust the header names slightly after pasting if they don't match exactly.

## Input Data Format Requirements

Ensure your input CSV file (e.g., the one generated from `songdata.io` and saved) has at least the following columns:

*   `Track`: The name/title of the track (string).
*   `Camelot`: The Camelot key (e.g., '6A', '12B', '7A') (string).
*   `BPM`: Beats Per Minute (numeric).
*   `Energy`: Energy level (numeric, often 0-1 or 0-10, the script assumes higher is more energetic).

**Important:** The values in the `BPM` and `Energy` columns *must* be numeric. Text or missing values in these columns will cause errors or lead to tracks being dropped.

Optional columns used:

*   `Popularity`: Track popularity (numeric, e.g., 0-100) used for finding the opener.

Other columns like `Duration`, `Key`, `Acousticness`, etc., will be kept in the output file but are not directly used in the sorting logic defined here.

## Using the PlaylistSorter in the Notebook

Instead of using command-line arguments, we'll set the configuration variables directly in the next cell and then instantiate and use the `PlaylistSorter`.

In [4]:
# --- Configuration --- 
# <<< CHANGE THESE VALUES >>>
input_csv_file = 'unsorted_input.csv'  # IMPORTANT: Replace with the actual path to your CSV file
output_csv_file = 'sorted_output.csv' # Name for the output file

# Set to a specific 0-based index (e.g., 0, 1, 2...) to force start track,
# or set to None to let the script find the best opener automatically.
start_track_index = None 

show_verbose_path = True # Set to True to print the transition paths, False to hide them
# ---------------------

# --- Execution ---
print(f"Attempting to sort playlist from: {input_csv_file}")

try:
    # 1. Initialize the sorter (loads data)
    sorter = PlaylistSorter(input_csv_file)
    
    # Check if initialization was successful (tracks were loaded)
    if sorter.tracks is not None and not sorter.tracks.empty:
        print(f"Successfully loaded {len(sorter.tracks)} tracks.")
        
        # 2. Create the sorted list of indices
        print("Sorting playlist...")
        sorted_indices = sorter.create_sorted_playlist(start_track_index)
        
        if sorted_indices is not None:
            print(f"Sorting complete. Playlist has {len(sorted_indices)} tracks.")
            
            # 3. Export the sorted playlist
            sorter.export_sorted_playlist(sorted_indices, output_csv_file)
            
            # 4. Optionally print the transition path
            if show_verbose_path:
                sorter.print_transition_path(sorted_indices)
        else:
            print("Could not generate sorted playlist indices.")
            
    else:
        # Error message already printed by _load_tracks or __init__
        print("Playlist sorting aborted due to initialization errors.")

except FileNotFoundError:
    # This might be redundant if _load_tracks handles it, but good as a fallback
    print(f"Error: Input file not found at '{input_csv_file}'")
    print("Please ensure the file exists and the path is correct in the configuration cell.")
except ValueError as ve:
    # Catch ValueErrors raised during initialization or processing
    print(f"Configuration or Data Error: {ve}")
except Exception as e:
    print(f"An unexpected error occurred during notebook execution: {e}")
    # You might want to print traceback for debugging
    # import traceback
    # traceback.print_exc()

Attempting to sort playlist from: unsorted_input.csv
Successfully loaded 322 tracks.
Sorting playlist...
Selected opener: Track 'Dekha Hazaro Dafaa' (Index: 302) with score 6.00
Transition: Dekha Hazaro Dafaa -> Banjaara (via Primary Key Match)
Transition: Banjaara -> Javeda Zindagi-Tose Naina Lage (via Primary Key Match)
Transition: Javeda Zindagi-Tose Naina Lage -> Saware (via Primary Key Match)
Transition: Saware -> Saware (From "Phantom") (via Primary Key Match)
Transition: Saware (From "Phantom") -> Tere Naina (via Primary Key Match)
Transition: Tere Naina -> Chale Aana (From "De De Pyaar De") (via Primary Key Match)
Transition: Chale Aana (From "De De Pyaar De") -> Ishq Shava (via Primary Key Match)
Transition: Ishq Shava -> Hasi - Male Version (via Primary Key Match)
Transition: Hasi - Male Version -> Hua Hain Aaj Pehli Baar (via Primary Key Match)
Transition: Hua Hain Aaj Pehli Baar -> Tujhko Jo Paaya (From "Crook") (via Primary Key Match)
Transition: Tujhko Jo Paaya (From "Cro